# Inserting ISS Data To MySQL with SQLAlchemy

## Collecting data from the API

In [12]:
API_key = 'b307fb60571f3805fce4f710d609ba0b'

In [13]:
import requests
url =  "http://api.open-notify.org/iss-now.json"
response = requests.get(url)
json_response = response.json()
json_response

{'timestamp': 1672992635,
 'message': 'success',
 'iss_position': {'latitude': '-16.9134', 'longitude': '167.9882'}}

Extract only the pieces of info we need and store them in a dictionary

In [14]:
data_to_insert = {"latitude":[json_response["iss_position"]["latitude"]],
                  "longitude":[json_response["iss_position"]["longitude"]],
                  "iss_timestamp":[json_response["timestamp"]]}
data_to_insert

{'latitude': ['-16.9134'],
 'longitude': ['167.9882'],
 'iss_timestamp': [1672992635]}

Create a pandas Dataframe with the same columns as the SQL table we want to have in our database

In [15]:
import pandas as pd

iss_df = pd.DataFrame.from_dict(data_to_insert)
iss_df

latitude longitude  iss_timestamp
0  -16.9134  167.9882     1672992635

Timestamp from UNIX to proper datetime format

In [16]:
iss_df.iss_timestamp = pd.to_datetime(iss_df.iss_timestamp,unit='s')
iss_df

latitude longitude       iss_timestamp
0  -16.9134  167.9882 2023-01-06 08:10:35

In [18]:
data_to_insert = {"iss_timestamp":[json_response["timestamp"]],
                  "activity_status":["active"]}
status_df = pd.DataFrame.from_dict(data_to_insert)
status_df.iss_timestamp = pd.to_datetime(status_df.iss_timestamp,unit='s')
status_df

iss_timestamp activity_status
0 2023-01-06 08:10:35          active

# SQLAlchemy

In [19]:
# pip install sqlalchemy 
import sqlalchemy # install if needed

Specify MySQL connection.

Before this part you should have already created a schema (database) `sql_workshop` on you local mySql server with coresponding tables `iss_logs` and `iss_status`.

Data that we want to insert into a ddatabase should follow the same structure as a table in a database.

In [20]:
schema="sql_workshop"   # name of the database you want to use here
host="127.0.0.1"        # to connect to your local server
user="root"
password="root" # your password!!!!  
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
#con = f'mysql+pymysql://{user}:@{host}:{port}/{schema}'

Use pandas method `to_sql` with the argument `if_exists=append` to create the table (only the first time we run it) and insert the new rows into it.

In [21]:
status_df

iss_timestamp activity_status
0 2023-01-06 08:10:35          active

In [22]:
!pip install pymysql

In [23]:
status_df.to_sql('iss_status',con=con,if_exists='append',index=False)

1

Check on MySQLWorkbench that a new table `iss_status` exists within the `iss_workshop` database, and that a new row has been inserted on it. If you run the whole notebook again, another row should appear there.

We can now populate the other table `iss_logs` by giving a fiction status active to the same timestamp.

In [51]:
iss_df

latitude longitude       iss_timestamp
0  -27.2641  -59.6832 2023-01-05 22:49:55

In [52]:
iss_df.to_sql('iss_logs',         # 'iss_logs'-> table name;
              if_exists='append', # if_exists -> will create new table if doesn't exist, otherwise, 'append' - will append data to existing table;
              con=con,            # con-> connection string;
              index=False)        # index = False -> will not send index column to database                                          

1

In [53]:
table_df = pd.read_sql_table(
    'iss_logs',
    con=con
    )

In [54]:
table_df

id  latitude  longitude       iss_timestamp
0   1  -27.2641   -59.6832 2023-01-05 22:49:55